# 1. 神经网络绪论
神经网络是一种模仿生物神经网络（动物的中枢神经系统，特别是大脑）的结构和功能的数学模型或计算模型，用于对函数进行估计或近似。神经网络由大量的人工神经元联结进行计算。


## 1.1 与传统的机器学习方法相比
区别在于：神经网络没有固定形式，可以DIY。这也决定了神经网络可以解决各种特定问题，只要我们能够针对这个特定的问题，想出一种合理的网络结构与优化目标，就可以解决该问题。这也是目前神经网络创新的主要方向。



## 1.2 初次使用
通过pip安装 torch

In [1]:
! pip install -i https://mirrors.ustc.edu.cn/pypi/web/simple torch 

Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple


# 2 从感知机到神经网络
感知器是Frank Rosenblatt在1957年就职于康奈尔航空实验室时所发明的一种人工神经网络。
它可以被视为一种最简单形式的前馈神经网络，是一种二元线性分类器。

## 2.1 利用感知机做二分类
感知机模型是一个特殊的单层神经网络，单层感知机的输出为1个神经元，在这里我们展示如何使用单层感知机完成二分类任务
注意！！！由于神经网络的特殊性，在输入到神经网络之前，需要对输入进行标准化

### 2.1.1 单层感知机做二分类

In [2]:
import torch
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics

# 加载乳腺癌数据
cancer =datasets.load_breast_cancer()
X=cancer.data
y=cancer.target

# 标准化过程（大家可以尝试注释一下两行比较标准化的重要性）
x_norm = np.linalg.norm(X,axis=0)
X = X/x_norm

print(X.shape)
print(y.shape)
# 数据划分
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, random_state=1)
X_tr, X_va, y_tr, y_va = train_test_split(X_tr, y_tr, test_size=0.25, random_state=1)

(569, 30)
(569,)


In [3]:
# 搭建网络
class Perceptron(torch.nn.Module):
    def __init__(self, input_dim):
        super(Perceptron, self).__init__()
        self.lin = torch.nn.Linear(input_dim, 1)
    
    def forward(self, x):
        # 重复使用x以减少现存开销
        x = self.lin(x)
        return x

In [4]:
# 构建模型
per = Perceptron(input_dim = 30)
# 定义二分类损失
loss_func = torch.nn.MSELoss()
# 定义优化器
optimizer = torch.optim.Adam(per.parameters(), lr=0.1)

In [5]:
# 构建evaluate 函数
def evaluate(net, X, y):
    # 将网络转化为测试模式（移除dropout等）
    net.eval()
    # 强制在验证阶段不保存梯度
    with torch.no_grad():
        # 通过网络输出结果
        out = net(torch.FloatTensor(X)).cpu().data.numpy()
        # 将连续数值转化为二分类问题，默认阈值选为0.5
        y_pred = np.zeros_like(out)
        y_pred[out>0.5] = 1
    fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=1)
    return metrics.auc(fpr, tpr)

In [6]:
# 测试evaluate 函数
evaluate(per,X_va,y_va)
# auc 为 0.5 意味着完全随机

0.5

In [7]:
for epoch in range(300):
    out = per(torch.FloatTensor(X_tr))
    loss = loss_func(out, torch.FloatTensor(y_tr).reshape([-1,1]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集与测试集指标
    va_score = evaluate(per,X_va,y_va)
    ts_score = evaluate(per,X_ts,y_ts)
    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 验证集分数:",round(va_score,5)," 测试集分数:",round(ts_score,5))

迭代次数: 0  损失: 0.65031  验证集分数: 0.5  测试集分数: 0.5
迭代次数: 10  损失: 0.24668  验证集分数: 0.525  测试集分数: 0.5
迭代次数: 20  损失: 0.15359  验证集分数: 0.8875  测试集分数: 0.76687
迭代次数: 30  损失: 0.10813  验证集分数: 0.91149  测试集分数: 0.79563
迭代次数: 40  损失: 0.09344  验证集分数: 0.9125  测试集分数: 0.77877
迭代次数: 50  损失: 0.08938  验证集分数: 0.91824  测试集分数: 0.85516
迭代次数: 60  损失: 0.08619  验证集分数: 0.925  测试集分数: 0.81448
迭代次数: 70  损失: 0.08308  验证集分数: 0.925  测试集分数: 0.8502
迭代次数: 80  损失: 0.08086  验证集分数: 0.91824  测试集分数: 0.86905
迭代次数: 90  损失: 0.07916  验证集分数: 0.91824  测试集分数: 0.85714
迭代次数: 100  损失: 0.07768  验证集分数: 0.91824  测试集分数: 0.85714
迭代次数: 110  损失: 0.07637  验证集分数: 0.91824  测试集分数: 0.85714
迭代次数: 120  损失: 0.07519  验证集分数: 0.93074  测试集分数: 0.85714
迭代次数: 130  损失: 0.07413  验证集分数: 0.93074  测试集分数: 0.85714
迭代次数: 140  损失: 0.07316  验证集分数: 0.93074  测试集分数: 0.86905
迭代次数: 150  损失: 0.07227  验证集分数: 0.95  测试集分数: 0.86905
迭代次数: 160  损失: 0.07146  验证集分数: 0.95  测试集分数: 0.86905
迭代次数: 170  损失: 0.0707  验证集分数: 0.95  测试集分数: 0.88095
迭代次数: 180  损失: 0.07001  验证集分数: 0.95  测试集分数: 0.88095


### 2.1.2 带激活函数感知机做二分类

In [9]:
# 搭建网络
class Perceptron(torch.nn.Module):
    def __init__(self, input_dim):
        super(Perceptron, self).__init__()
        self.lin = torch.nn.Linear(input_dim, 1)
        self.act = torch.nn.Sigmoid()
    
    def forward(self, x):
        # 重复使用x以减少现存开销
        x = self.lin(x)
        x = self.act(x)
        return x

In [10]:
# 构建模型
per = Perceptron(input_dim = 30)
# 定义二分类损失
loss_func = torch.nn.MSELoss()
# 定义优化器
optimizer = torch.optim.Adam(per.parameters(), lr=0.1)

In [12]:
for epoch in range(300):
    out = per(torch.FloatTensor(X_tr))
    loss = loss_func(out, torch.FloatTensor(y_tr).reshape([-1,1]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集与测试集指标
    va_score = evaluate(per,X_va,y_va)
    ts_score = evaluate(per,X_ts,y_ts)
    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 验证集分数:",round(va_score,5)," 测试集分数:",round(ts_score,5))

迭代次数: 0  损失: 0.10133  验证集分数: 0.92399  测试集分数: 0.80754
迭代次数: 10  损失: 0.09221  验证集分数: 0.92399  测试集分数: 0.83135
迭代次数: 20  损失: 0.08518  验证集分数: 0.93074  测试集分数: 0.83135
迭代次数: 30  损失: 0.07958  验证集分数: 0.93074  测试集分数: 0.85516
迭代次数: 40  损失: 0.07501  验证集分数: 0.93074  测试集分数: 0.85516
迭代次数: 50  损失: 0.07121  验证集分数: 0.94324  测试集分数: 0.85516
迭代次数: 60  损失: 0.068  验证集分数: 0.94324  测试集分数: 0.90278
迭代次数: 70  损失: 0.06525  验证集分数: 0.95  测试集分数: 0.90278
迭代次数: 80  损失: 0.06285  验证集分数: 0.95  测试集分数: 0.91468
迭代次数: 90  损失: 0.06074  验证集分数: 0.95  测试集分数: 0.91468
迭代次数: 100  损失: 0.05887  验证集分数: 0.95  测试集分数: 0.92659
迭代次数: 110  损失: 0.05718  验证集分数: 0.95  测试集分数: 0.92659
迭代次数: 120  损失: 0.05565  验证集分数: 0.95  测试集分数: 0.92659
迭代次数: 130  损失: 0.05426  验证集分数: 0.95  测试集分数: 0.93353
迭代次数: 140  损失: 0.05298  验证集分数: 0.95  测试集分数: 0.93353
迭代次数: 150  损失: 0.0518  验证集分数: 0.95  测试集分数: 0.93353
迭代次数: 160  损失: 0.0507  验证集分数: 0.95  测试集分数: 0.93353
迭代次数: 170  损失: 0.04968  验证集分数: 0.95  测试集分数: 0.93353
迭代次数: 180  损失: 0.04872  验证集分数: 0.9625  测试集分数: 0.94544
迭代次数

### 2.1.2 多层感知机做二分类

In [13]:
# 搭建网络
class MLP(torch.nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.lin1 = torch.nn.Linear(input_dim, 64)
        self.act1 = torch.nn.Sigmoid()
        self.lin2 = torch.nn.Linear(64, 1)
        self.act2 = torch.nn.Sigmoid()
    
    def forward(self, x):
        # 重复使用x以减少现存开销
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        return x

In [14]:
# 构建模型
mlp = MLP(input_dim = 30)
# 定义二分类损失
loss_func = torch.nn.MSELoss()
# 定义优化器
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.1)

In [15]:
for epoch in range(300):
    out = mlp(torch.FloatTensor(X_tr))
    loss = loss_func(out, torch.FloatTensor(y_tr).reshape([-1,1]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集与测试集指标
    va_score = evaluate(mlp,X_va,y_va)
    ts_score = evaluate(mlp,X_ts,y_ts)
    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 验证集分数:",round(va_score,5)," 测试集分数:",round(ts_score,5))

迭代次数: 0  损失: 0.29847  验证集分数: 0.5  测试集分数: 0.5
迭代次数: 10  损失: 0.34352  验证集分数: 0.5  测试集分数: 0.5
迭代次数: 20  损失: 0.22963  验证集分数: 0.9277  测试集分数: 0.84921
迭代次数: 30  损失: 0.17473  验证集分数: 0.55  测试集分数: 0.52381
迭代次数: 40  损失: 0.13277  验证集分数: 0.91149  测试集分数: 0.80556
迭代次数: 50  损失: 0.09492  验证集分数: 0.92399  测试集分数: 0.8244
迭代次数: 60  损失: 0.0702  验证集分数: 0.92399  测试集分数: 0.87202
迭代次数: 70  损失: 0.05556  验证集分数: 0.94899  测试集分数: 0.92659
迭代次数: 80  损失: 0.04624  验证集分数: 0.96149  测试集分数: 0.93849
迭代次数: 90  损失: 0.04169  验证集分数: 0.96824  测试集分数: 0.94544
迭代次数: 100  损失: 0.03855  验证集分数: 0.96824  测试集分数: 0.94544
迭代次数: 110  损失: 0.03587  验证集分数: 0.975  测试集分数: 0.94544
迭代次数: 120  损失: 0.03349  验证集分数: 0.9875  测试集分数: 0.94544
迭代次数: 130  损失: 0.03133  验证集分数: 0.9875  测试集分数: 0.94544
迭代次数: 140  损失: 0.02936  验证集分数: 0.9875  测试集分数: 0.94544
迭代次数: 150  损失: 0.02754  验证集分数: 0.9875  测试集分数: 0.94544
迭代次数: 160  损失: 0.02587  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 170  损失: 0.02434  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 180  损失: 0.02294  验证集分数: 0.9875  测试集分数: 0.964

# 3 神经网络激活函数
激活函数（Activation Function）是一种添加到人工神经网络中的函数，旨在帮助网络学习数据中的复杂模式。 类似于人类大脑中基于神经元的模型，激活函数最终决定了要发射给下一个神经元的内容。 在人工神经网络中，一个节点的激活函数定义了该节点在给定的输入或输入集合下的输出。

### 3.1 激活函数为Tanh+Sigmoid

In [23]:
# 搭建网络
class MLP(torch.nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.lin1 = torch.nn.Linear(input_dim, 64)
        self.act1 = torch.nn.Tanh()
        self.lin2 = torch.nn.Linear(64, 1)
        self.act2 = torch.nn.Sigmoid()
    
    def forward(self, x):
        # 重复使用x以减少现存开销
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        return x

In [24]:
# 构建模型
per = MLP(input_dim = 30)
# 定义二分类损失
loss_func = torch.nn.MSELoss()
# 定义优化器
optimizer = torch.optim.Adam(per.parameters(), lr=0.1)

In [25]:
for epoch in range(300):
    out = per(torch.FloatTensor(X_tr))
    loss = loss_func(out, torch.FloatTensor(y_tr).reshape([-1,1]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集与测试集指标
    va_score = evaluate(per,X_va,y_va)
    ts_score = evaluate(per,X_ts,y_ts)
    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 验证集分数:",round(va_score,5)," 测试集分数:",round(ts_score,5))

迭代次数: 0  损失: 0.25529  验证集分数: 0.5  测试集分数: 0.5
迭代次数: 10  损失: 0.05671  验证集分数: 0.95372  测试集分数: 0.91766
迭代次数: 20  损失: 0.03884  验证集分数: 0.96824  测试集分数: 0.94544
迭代次数: 30  损失: 0.0289  验证集分数: 0.96824  测试集分数: 0.94544
迭代次数: 40  损失: 0.02199  验证集分数: 0.975  测试集分数: 0.95238
迭代次数: 50  损失: 0.01887  验证集分数: 0.9875  测试集分数: 0.96429
迭代次数: 60  损失: 0.01684  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 70  损失: 0.01545  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 80  损失: 0.01445  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 90  损失: 0.0134  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 100  损失: 0.01103  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 110  损失: 0.01018  验证集分数: 0.975  测试集分数: 0.97619
迭代次数: 120  损失: 0.00965  验证集分数: 0.9875  测试集分数: 0.96429
迭代次数: 130  损失: 0.00931  验证集分数: 0.9875  测试集分数: 0.96429
迭代次数: 140  损失: 0.00898  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 150  损失: 0.0087  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 160  损失: 0.00845  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 170  损失: 0.00822  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 180  损失: 0.00801  验证集分数: 0.9875  测试集分数: 0.

### 3.2 激活函数为ReLU+Sigmoid

In [19]:
# 搭建网络
class MLP(torch.nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.lin1 = torch.nn.Linear(input_dim, 64)
        self.act1 = torch.nn.ReLU()
        self.lin2 = torch.nn.Linear(64, 1)
        self.act2 = torch.nn.Sigmoid()
    
    def forward(self, x):
        # 重复使用x以减少现存开销
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        return x

In [20]:
# 构建模型
per = MLP(input_dim = 30)
# 定义二分类损失
loss_func = torch.nn.MSELoss()
# 定义优化器
optimizer = torch.optim.Adam(per.parameters(), lr=0.1)

In [22]:
for epoch in range(300):
    out = per(torch.FloatTensor(X_tr))
    loss = loss_func(out, torch.FloatTensor(y_tr).reshape([-1,1]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集与测试集指标
    va_score = evaluate(per,X_va,y_va)
    ts_score = evaluate(per,X_ts,y_ts)
    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 验证集分数:",round(va_score,5)," 测试集分数:",round(ts_score,5))

迭代次数: 0  损失: 0.01921  验证集分数: 0.9875  测试集分数: 0.96429
迭代次数: 10  损失: 0.01685  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 20  损失: 0.01418  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 30  损失: 0.01241  验证集分数: 0.9875  测试集分数: 0.97619
迭代次数: 40  损失: 0.01145  验证集分数: 0.975  测试集分数: 0.96429
迭代次数: 50  损失: 0.01091  验证集分数: 0.9875  测试集分数: 0.96429
迭代次数: 60  损失: 0.01046  验证集分数: 0.9875  测试集分数: 0.96429
迭代次数: 70  损失: 0.01007  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 80  损失: 0.00973  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 90  损失: 0.00943  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 100  损失: 0.00915  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 110  损失: 0.0089  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 120  损失: 0.00891  验证集分数: 0.975  测试集分数: 0.95734
迭代次数: 130  损失: 0.00854  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 140  损失: 0.00833  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 150  损失: 0.0082  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 160  损失: 0.00803  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 170  损失: 0.00788  验证集分数: 0.9875  测试集分数: 0.95734
迭代次数: 180  损失: 0.00775  验证集分数: 0.9875  测试集分

# 4 神经网络损失函数


## 4.1 神经网络反向传播

In [498]:
# 搭建网络
class MLP(torch.nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.lin1 = torch.nn.Linear(input_dim, 64)
        self.act1 = torch.nn.ReLU()
        self.lin2 = torch.nn.Linear(64, 1)
        self.act2 = torch.nn.Sigmoid()
    
    def forward(self, x):
        # 重复使用x以减少现存开销
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        return x

In [499]:
# 构建模型
per = MLP(input_dim = 30)
# 定义二分类损失
loss_func = torch.nn.MSELoss()
# 定义优化器
optimizer = torch.optim.Adam(per.parameters(), lr=0.1)

In [500]:
# 测试模型是否定义正确
# 注意PyTorch需要提前将输入转化为Tensor形式
out = per(torch.FloatTensor(X_tr[:10]))


In [501]:
# 计算Loss
# 注意PyTorch需要提前将输入转化为Tensor形式
loss = loss_func(out, torch.FloatTensor(y_tr[:10]).reshape([-1,1]))
print(loss)

tensor(0.2611, grad_fn=<MseLossBackward>)


In [502]:
# 尝试反向传播Loss
# 首先清空上一步的梯度
optimizer.zero_grad()
# 反向传播Loss
loss.backward()
# 通过反向传播的Loss优化参数
optimizer.step()
# 打印梯度
for name, parms in per.named_parameters():
    print('-->name:', name, '-->grad_requirs:',parms.requires_grad, \
    ' -->grad_value:',parms.grad)

-->name: lin1.weight -->grad_requirs: True  -->grad_value: tensor([[-2.0431e-04, -2.0868e-04, -1.9664e-04,  ..., -3.6695e-05,
         -2.3156e-04, -2.1938e-04],
        [-1.5774e-05, -1.5384e-05, -1.8173e-05,  ..., -7.5233e-05,
         -4.9994e-05, -5.0309e-05],
        [ 3.7911e-04,  3.7293e-04,  3.6055e-04,  ..., -2.8746e-05,
          3.7697e-04,  3.5433e-04],
        ...,
        [ 2.8349e-05,  2.7886e-05,  2.6961e-05,  ..., -2.1495e-06,
          2.8189e-05,  2.6496e-05],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
-->name: lin1.bias -->grad_requirs: True  -->grad_value: tensor([-6.7287e-03,  1.3682e-04,  1.3176e-02,  1.9708e-02,  3.7581e-03,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.8422e-02,
         2.4182e-03, -7.0067e-03, -7.7396e-03, -1.0954e-03,  1.4173e-02,
        -4.0671e-03,  3.7423e-05,

## 4.2 神经网络损失函数

### 4.2.1 损失函数为交叉熵损失函数

In [155]:
from sklearn.datasets import load_digits
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
 
# 构建数据加载器 
digits =datasets.load_digits()
X=digits.data
y=digits.target

print(X.shape)
print(y.shape)
# 数据划分
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, random_state=0)

(1797, 64)
(1797,)


In [156]:
# 搭建网络
class HMLP(nn.Module):
    def __init__(self, input_dim=64, output_dim=10):
        super(HMLP, self).__init__()
        self.lin1 = nn.Linear(input_dim, 50)
        self.lin2 = nn.Linear(50, 40)
        self.lin3 = nn.Linear(40, 30)
        self.lin4 = nn.Linear(30, output_dim)
        self.act = nn.ReLU()
    
    def forward(self, x):
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        x = self.lin4(x)
        x = self.act(x)
        return x

In [157]:
# 构建模型
per = HMLP(input_dim = 64)
loss_func = nn.CrossEntropyLoss(reduction="mean")
# 定义优化器
optimizer = torch.optim.Adam(per.parameters(), lr=0.01)

In [158]:
for epoch in range(160):
    out = per(torch.Tensor(X_tr))
    loss = loss_func(out, torch.LongTensor(y_tr))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(y_ts)):
            res = per(torch.Tensor(X_ts))
            y_ts = torch.Tensor(y_ts)
            _, pred = torch.max(res.data, dim=1)
            total += len(y_ts)
            correct += (pred == y_ts).sum().item()
    acc = correct/total

    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 准确率:",round(acc,5))

迭代次数: 0  损失: 2.32587  准确率: 0.16111
迭代次数: 10  损失: 0.83983  准确率: 0.83889
迭代次数: 20  损失: 0.20071  准确率: 0.93056
迭代次数: 30  损失: 0.10795  准确率: 0.95278
迭代次数: 40  损失: 0.06872  准确率: 0.95556
迭代次数: 50  损失: 0.04497  准确率: 0.95556
迭代次数: 60  损失: 0.03186  准确率: 0.95278
迭代次数: 70  损失: 0.02449  准确率: 0.96111
迭代次数: 80  损失: 0.01958  准确率: 0.96111
迭代次数: 90  损失: 0.0163  准确率: 0.96111
迭代次数: 100  损失: 0.01406  准确率: 0.96111
迭代次数: 110  损失: 0.01242  准确率: 0.96111
迭代次数: 120  损失: 0.01121  准确率: 0.96111
迭代次数: 130  损失: 0.0103  准确率: 0.96111
迭代次数: 140  损失: 0.00961  准确率: 0.96111
迭代次数: 150  损失: 0.00908  准确率: 0.95556
迭代次数: 160  损失: 0.00867  准确率: 0.95556
迭代次数: 170  损失: 0.00834  准确率: 0.95556
迭代次数: 180  损失: 0.00808  准确率: 0.95556
迭代次数: 190  损失: 0.00787  准确率: 0.95556
迭代次数: 200  损失: 0.0077  准确率: 0.95556
迭代次数: 210  损失: 0.00755  准确率: 0.95556
迭代次数: 220  损失: 0.00744  准确率: 0.95556
迭代次数: 230  损失: 0.00733  准确率: 0.95556
迭代次数: 240  损失: 0.00725  准确率: 0.95833
迭代次数: 250  损失: 0.00717  准确率: 0.95833
迭代次数: 260  损失: 0.00711  准确率: 0.95833
迭代次数: 270  损失: 

# 5 神经网络优化器

## 5.1 优化器为SGD

In [149]:
# 搭建网络
class HMLP(nn.Module):
    def __init__(self, input_dim=64, output_dim=10):
        super(HMLP, self).__init__()
        self.lin1 = nn.Linear(input_dim, 50)
        self.lin2 = nn.Linear(50, 40)
        self.lin3 = nn.Linear(40, 30)
        self.lin4 = nn.Linear(30, output_dim)
        self.act = nn.ReLU()
    
    def forward(self, x):
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        x = self.lin4(x)
        x = self.act(x)
        return x

In [150]:
# 构建模型
per = HMLP(input_dim = 64)
loss_func = nn.CrossEntropyLoss(reduction="mean")
# 定义优化器
optimizer = torch.optim.SGD(per.parameters(), lr=0.01)

In [151]:
for epoch in range(160):
    out = per(torch.Tensor(X_tr))
    loss = loss_func(out, torch.LongTensor(y_tr))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(y_ts)):
            res = per(torch.Tensor(X_ts))
            y_ts = torch.Tensor(y_ts)
            _, pred = torch.max(res.data, dim=1)
            total += len(y_ts)
            correct += (pred == y_ts).sum().item()
    acc = correct/total

    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 准确率:",round(acc,5))

迭代次数: 0  损失: 2.40365  准确率: 0.12222
迭代次数: 10  损失: 2.07646  准确率: 0.37778
迭代次数: 20  损失: 1.90038  准确率: 0.48889
迭代次数: 30  损失: 1.75784  准确率: 0.51111
迭代次数: 40  损失: 1.64091  准确率: 0.525
迭代次数: 50  损失: 1.54682  准确率: 0.53611
迭代次数: 60  损失: 1.47248  准确率: 0.54167
迭代次数: 70  损失: 1.41285  准确率: 0.55556
迭代次数: 80  损失: 1.35279  准确率: 0.58056
迭代次数: 90  损失: 1.28353  准确率: 0.59722
迭代次数: 100  损失: 1.2235  准确率: 0.62222
迭代次数: 110  损失: 1.17738  准确率: 0.63056
迭代次数: 120  损失: 1.1362  准确率: 0.63611
迭代次数: 130  损失: 1.08013  准确率: 0.64722
迭代次数: 140  损失: 1.0283  准确率: 0.68056
迭代次数: 150  损失: 0.98949  准确率: 0.70556
迭代次数: 160  损失: 0.95771  准确率: 0.70833
迭代次数: 170  损失: 0.93029  准确率: 0.71111
迭代次数: 180  损失: 0.90604  准确率: 0.71944
迭代次数: 190  损失: 0.88407  准确率: 0.725
迭代次数: 200  损失: 0.86361  准确率: 0.73611
迭代次数: 210  损失: 0.84524  准确率: 0.75833
迭代次数: 220  损失: 0.82823  准确率: 0.76111
迭代次数: 230  损失: 0.81232  准确率: 0.76944
迭代次数: 240  损失: 0.79759  准确率: 0.77222
迭代次数: 250  损失: 0.78361  准确率: 0.77222
迭代次数: 260  损失: 0.77051  准确率: 0.775
迭代次数: 270  损失: 0.7581

### 5.2 优化器为RMSprop


In [152]:
# 搭建网络
class HMLP(nn.Module):
    def __init__(self, input_dim=64, output_dim=10):
        super(HMLP, self).__init__()
        self.lin1 = nn.Linear(input_dim, 50)
        self.lin2 = nn.Linear(50, 40)
        self.lin3 = nn.Linear(40, 30)
        self.lin4 = nn.Linear(30, output_dim)
        self.act = nn.ReLU()
    
    def forward(self, x):
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        x = self.lin4(x)
        x = self.act(x)
        return x

In [153]:
# 构建模型
per = HMLP(input_dim = 64)
loss_func = nn.CrossEntropyLoss(reduction="mean")
# 定义优化器
optimizer = torch.optim.RMSprop(per.parameters(), lr=0.01)

In [154]:
for epoch in range(160):
    out = per(torch.Tensor(X_tr))
    loss = loss_func(out, torch.LongTensor(y_tr))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算验证集
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(y_ts)):
            res = per(torch.Tensor(X_ts))
            y_ts = torch.Tensor(y_ts)
            _, pred = torch.max(res.data, dim=1)
            total += len(y_ts)
            correct += (pred == y_ts).sum().item()
    acc = correct/total

    if epoch % 10 == 0:
        print("迭代次数:",epoch," 损失:", float("%.5f" % loss.cpu().data.numpy())," 准确率:",round(acc,5))

迭代次数: 0  损失: 2.41154  准确率: 0.08889
迭代次数: 10  损失: 20.08726  准确率: 0.21111
迭代次数: 20  损失: 10.39424  准确率: 0.32222
迭代次数: 30  损失: 1.27526  准确率: 0.53056
迭代次数: 40  损失: 1.05481  准确率: 0.61389
迭代次数: 50  损失: 0.92066  准确率: 0.65278
迭代次数: 60  损失: 0.88758  准确率: 0.63056
迭代次数: 70  损失: 0.8356  准确率: 0.64167
迭代次数: 80  损失: 0.8096  准确率: 0.65
迭代次数: 90  损失: 0.78241  准确率: 0.65833
迭代次数: 100  损失: 0.76206  准确率: 0.66944
迭代次数: 110  损失: 0.73119  准确率: 0.7
迭代次数: 120  损失: 0.72762  准确率: 0.69444
迭代次数: 130  损失: 0.70508  准确率: 0.70833
迭代次数: 140  损失: 0.68948  准确率: 0.71667
迭代次数: 150  损失: 0.70041  准确率: 0.69444
迭代次数: 160  损失: 0.67685  准确率: 0.71667
迭代次数: 170  损失: 0.65502  准确率: 0.71944
迭代次数: 180  损失: 0.61843  准确率: 0.73611
迭代次数: 190  损失: 0.87469  准确率: 0.55833


KeyboardInterrupt: 